<a href="https://colab.research.google.com/github/xanny1111/Spring_2025-Plotnov-24-VMz-/blob/main/SadTalker_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
!rm -rf SadTalker
!git clone https://github.com/camenduru/SadTalker
%cd /content/SadTalker

!apt -y install aria2
!mkdir -p checkpoints
!mkdir -p checkpoints/BFM_Fitting

# Скачиваем необходимые файлы для BFM_Fitting
!wget -O checkpoints/BFM_Fitting/similarity_Lm3D_all.mat https://github.com/Winfredy/SadTalker/raw/main/checkpoints/BFM_Fitting/similarity_Lm3D_all.mat || true
!wget -O checkpoints/BFM_Fitting/Exp_Pca.bin https://github.com/Winfredy/SadTalker/raw/main/checkpoints/BFM_Fitting/Exp_Pca.bin || true
# Используем ПРАВИЛЬНЫЕ ссылки от camenduru
!wget -O checkpoints/epoch_20.pth https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2/epoch_20.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/SadTalker/resolve/main/new/checkpoints/mapping_00109-model.pth.tar -d /content/SadTalker/checkpoints -o mapping_00109-model.pth.tar
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/SadTalker/resolve/main/new/checkpoints/mapping_00229-model.pth.tar -d /content/SadTalker/checkpoints -o mapping_00229-model.pth.tar
!curl -L -o checkpoints/shape_predictor_68_face_landmarks.dat https://github.com/italojs/facial-landmarks-recognition/raw/master/shape_predictor_68_face_landmarks.dat

# Проверяем все файлы
!ls -lh checkpoints/
print("epoch_20.pth должен быть ~700MB, shape_predictor ~95MB")

!pip install -q torch torchvision torchaudio gradio

print("Модели скачаны! Применяем исправления...")

# Исправляем PyTorch совместимость для safetensors
!sed -i 's/checkpoint = torch.load(path_of_net_recon_model, map_location=torch.device(device))/checkpoint = torch.load(path_of_net_recon_model, map_location=torch.device(device), weights_only=False)/g' src/utils/preprocess.py

# Исправляем face_alignment
!sed -i 's/LandmarksType._2D/LandmarksType.TWO_D/g' src/face3d/extract_kp_videos.py

# ИСПРАВЛЯЕМ ПУТЬ К МОДЕЛИ - ДОБАВЬТЕ ЭТУ СТРОЧКУ
!sed -i 's/path_of_net_recon_model = .*/path_of_net_recon_model = "checkpoints\/epoch_20.pth"/g' src/gradio_demo.py

# Добавляем share=True
!sed -i 's/launch()/launch(share=True)/g' app.py

print("Готово! Запускаем интерфейс...")
!python app.py

Cloning into 'SadTalker'...
remote: Enumerating objects: 1632, done.
remote: Counting objects: 100% (561/561), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 1632 (delta 485), reused 457 (delta 457), pack-reused 1071 (from 1)
Receiving objects: 100% (1632/1632), 92.21 MiB | 31.55 MiB/s, done.
Resolving deltas: 100% (899/899), done.
/content/SadTalker
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
--2025-11-11 19:03:43--  https://github.com/Winfredy/SadTalker/raw/main/checkpoints/BFM_Fitting/similarity_Lm3D_all.mat
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/OpenTalker/SadTalker/raw/main/checkpoints/BFM_Fitting/similarity_Lm3D_all.mat [foll

In [63]:
!rm -f checkpoints/epoch_20_converted.pth

In [64]:
!rm -f checkpoints/SadTalker_V0.0.2_256.safetensors

In [69]:
!ls -lh checkpoints/

total 668M
-rw-r--r-- 1 root root 276M Apr  8  2023 epoch_20.pth
-rw-r--r-- 1 root root    0 Nov 11 17:56 facevid2vid_00189-model.pth.tar
drwxr-xr-x 3 root root 4.0K Nov 11 17:27 hub
-rw-r--r-- 1 root root 149M May 31  2023 mapping_00109-model.pth.tar
-rw-r--r-- 1 root root 149M May 31  2023 mapping_00229-model.pth.tar
drwxr-xr-x 7 root root 4.0K Nov 11 18:08 SadTalker
-rw-r--r-- 1 root root  96M Nov 11 18:38 shape_predictor_68_face_landmarks.dat


In [66]:
import torch

# Проверим структуру правильного файла
checkpoint = torch.load("checkpoints/epoch_20.pth", map_location='cpu')
print("Ключи в правильном файле:")
for k in checkpoint.keys():
    print(f"  - {k}")

# Проверим наличие net_recon
if 'net_recon' in checkpoint:
    print("✅ net_recon найден!")
    net_recon = checkpoint['net_recon']
    print(f"Тип: {type(net_recon)}")
    if isinstance(net_recon, dict):
        print(f"Количество ключей: {len(net_recon)}")
        print("Примеры ключей:")
        for i, k in enumerate(list(net_recon.keys())[:5]):
            print(f"  - {k}")
else:
    print("❌ net_recon не найден!")

Ключи в правильном файле:
  - mapping
  - optimizer_mapping
  - discriminator
  - optimizer_discriminator
  - epoch
❌ net_recon не найден!


In [67]:
!rm checkpoints/epoch_20.pth


In [58]:
import torch
from safetensors import safe_open

# Сначала посмотрим структуру через safetensors
try:
    with safe_open("checkpoints/epoch_20.pth", framework="pt") as f:
        print("Ключи в safetensors:")
        for k in f.keys():
            print(f"  - {k}")

        # Сохраняем все тензоры
        state_dict = {k: f.get_tensor(k) for k in f.keys()}
        torch.save({'net_recon': state_dict}, "checkpoints/epoch_20_converted.pth")
        print("Успешно сохранено через safetensors")

except Exception as e:
    print(f"Ошибка safetensors: {e}")

Ключи в safetensors:
  - audio2exp.audio_encoder.0.conv_block.0.bias
  - audio2exp.audio_encoder.0.conv_block.0.weight
  - audio2exp.audio_encoder.0.conv_block.1.bias
  - audio2exp.audio_encoder.0.conv_block.1.num_batches_tracked
  - audio2exp.audio_encoder.0.conv_block.1.running_mean
  - audio2exp.audio_encoder.0.conv_block.1.running_var
  - audio2exp.audio_encoder.0.conv_block.1.weight
  - audio2exp.audio_encoder.1.conv_block.0.bias
  - audio2exp.audio_encoder.1.conv_block.0.weight
  - audio2exp.audio_encoder.1.conv_block.1.bias
  - audio2exp.audio_encoder.1.conv_block.1.num_batches_tracked
  - audio2exp.audio_encoder.1.conv_block.1.running_mean
  - audio2exp.audio_encoder.1.conv_block.1.running_var
  - audio2exp.audio_encoder.1.conv_block.1.weight
  - audio2exp.audio_encoder.10.conv_block.0.bias
  - audio2exp.audio_encoder.10.conv_block.0.weight
  - audio2exp.audio_encoder.10.conv_block.1.bias
  - audio2exp.audio_encoder.10.conv_block.1.num_batches_tracked
  - audio2exp.audio_encode

In [59]:
!sed -i 's/path_of_net_recon_model = .*/path_of_net_recon_model = "checkpoints\/epoch_20_converted.pth"/g' src/gradio_demo.py

In [60]:
import torch

# Проверим структуру конвертированного файла
checkpoint = torch.load("checkpoints/epoch_20_converted.pth", map_location='cpu')
print("Ключи в конвертированном файле:")
for k in checkpoint.keys():
    print(f"  - {k}")

# Проверим наличие net_recon
if 'net_recon' in checkpoint:
    print("✅ net_recon найден в конвертированном файле!")
    print(f"Тип net_recon: {type(checkpoint['net_recon'])}")
    if isinstance(checkpoint['net_recon'], dict):
        print(f"Количество ключей в net_recon: {len(checkpoint['net_recon'])}")
else:
    print("❌ net_recon не найден!")

Ключи в конвертированном файле:
  - net_recon
✅ net_recon найден в конвертированном файле!
Тип net_recon: <class 'dict'>
Количество ключей в net_recon: 1305


In [54]:
!pip install safetensors
import torch
from safetensors import safe_open

with safe_open("checkpoints/epoch_20.pth", framework="pt") as f:
    state_dict = {k: f.get_tensor(k) for k in f.keys()}
torch.save(state_dict, "checkpoints/epoch_20_converted.pth")

# Обновляем путь в коде
!sed -i 's/path_of_net_recon_model = .*/path_of_net_recon_model = "checkpoints\/epoch_20_converted.pth"/g' src/gradio_demo.py

In [48]:
%cd /content/SadTalker/checkpoints

!ls -la
print("Должны быть 4 файла:")
print("- epoch_20.pth (~700MB)")
print("- shape_predictor_68_face_landmarks.dat (~95MB)")
print("- mapping_00109-model.pth.tar")
print("- mapping_00229-model.pth.tar")

/content/SadTalker/checkpoints
total 1720184
drwxr-xr-x 3 root root      4096 Nov 11 17:55 .
drwxr-xr-x 9 root root      4096 Nov 11 18:06 ..
-rw-r--r-- 1 root root 725066984 May 31  2023 epoch_20.pth
-rw-r--r-- 1 root root         0 Nov 11 17:56 facevid2vid_00189-model.pth.tar
drwxr-xr-x 3 root root      4096 Nov 11 17:27 hub
-rw-r--r-- 1 root root 155779231 May 31  2023 mapping_00109-model.pth.tar
-rw-r--r-- 1 root root 155521183 May 31  2023 mapping_00229-model.pth.tar
-rw-r--r-- 1 root root 725066984 May 31  2023 SadTalker_V0.0.2_256.safetensors
-rw-r--r-- 1 root root         0 Nov 11 18:06 shape_predictor_68_face_landmarks.dat
Должны быть 4 файла:
- epoch_20.pth (~700MB)
- shape_predictor_68_face_landmarks.dat (~95MB)
- mapping_00109-model.pth.tar
- mapping_00229-model.pth.tar


In [42]:
%cd /content/SadTalker

# Удаляем поврежденный файл
!rm -f checkpoints/shape_predictor_68_face_landmarks.dat

# Скачиваем с альтернативного надежного источника
!wget -O checkpoints/shape_predictor_68_face_landmarks.dat https://github.com/italojs/facial-landmarks-recognition/raw/master/shape_predictor_68_face_landmarks.dat

# Проверяем размер файла (должен быть ~95MB)
!ls -lh checkpoints/shape_predictor_68_face_landmarks.dat

print("Файл перекачан!")

/content/SadTalker
--2025-11-11 17:55:12--  https://github.com/italojs/facial-landmarks-recognition/raw/master/shape_predictor_68_face_landmarks.dat
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/italojs/facial-landmarks-recognition/master/shape_predictor_68_face_landmarks.dat [following]
--2025-11-11 17:55:12--  https://raw.githubusercontent.com/italojs/facial-landmarks-recognition/master/shape_predictor_68_face_landmarks.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99693937 (95M) [application/octet-stream]
Saving to: ‘checkpoints/shape_predictor_68_face_landmarks.dat’

checkpoints/shap

In [37]:
%cd /content/SadTalker

# Восстанавливаем исходный файл
!git checkout HEAD -- src/utils/preprocess.py
!git checkout HEAD -- src/facerender/animate.py
!git checkout HEAD -- src/audio2pose_models/audio2pose.py

print("Файлы восстановлены!")

/content/SadTalker
Файлы восстановлены!


In [38]:
%cd /content/SadTalker

# Исправляем только конкретные строки в preprocess.py
!sed -i 's/checkpoint = torch.load(path_of_net_recon_model, map_location=torch.device(device))/checkpoint = torch.load(path_of_net_recon_model, map_location=torch.device(device), weights_only=False)/g' src/utils/preprocess.py

print("preprocess.py исправлен!")

/content/SadTalker
preprocess.py исправлен!


In [39]:
%cd /content/SadTalker

!grep -A2 -B2 "torch.load" src/utils/preprocess.py
print("Проверка завершена!")

/content/SadTalker
        self.kp_extractor = KeypointExtractor(device)
        self.net_recon = networks.define_net_recon(net_recon='resnet50', use_last_fc=False, init_path='').to(device)
        checkpoint = torch.load(path_of_net_recon_model, map_location=torch.device(device), weights_only=False)    
        self.net_recon.load_state_dict(checkpoint['net_recon'])
        self.net_recon.eval()
Проверка завершена!


In [30]:
# Ячейка 1: Создаем исправленный app.py без TTS
%cd /content/SadTalker

app_content = '''
import os
import gradio as gr
from src.gradio_demo import SadTalker

def sadtalker_demo():
    sad_talker = SadTalker(lazy_load=True)

    with gr.Blocks(analytics_enabled=False) as sadtalker_interface:
        gr.Markdown("<div align='center'> <h2> 😭 SadTalker</h2></div>")

        with gr.Row():
            with gr.Column(variant='panel'):
                with gr.Tabs(elem_id="sadtalker_source_image"):
                    with gr.TabItem('Upload image'):
                        with gr.Row():
                            source_image = gr.Image(label="Source image", source="upload", type="filepath", elem_id="img2img_image")

                with gr.Tabs(elem_id="sadtalker_input_audio"):
                    with gr.TabItem('Upload audio'):
                        with gr.Row():
                            driven_audio = gr.Audio(label="Input audio", source="upload", type="filepath")

            with gr.Column(variant='panel'):
                with gr.Tabs(elem_id="sadtalker_checkbox"):
                    with gr.TabItem('Settings'):
                        with gr.Column(variant='panel'):
                            with gr.Row():
                                still_mode = gr.Checkbox(label="Still Mode", value=True)
                                preprocess_type = gr.Radio(['crop', 'resize','full', 'extcrop', 'extfull'], value='crop', label='preprocess')
                            with gr.Row():
                                batch_size = gr.Slider(label="Batch size", step=1, maximum=10, value=2)
                                size_of_image = gr.Radio([256, 512], value=256, label='resolution')

                with gr.Tabs(elem_id="sadtalker_genearted"):
                    gen_video = gr.Video(label="Generated video", format="mp4", width=256)

        with gr.Row():
            submit = gr.Button('Generate', variant='primary')

        submit.click(
            fn=sad_talker.test,
            inputs=[source_image,
                    driven_audio,
                    still_mode,
                    preprocess_type,
                    batch_size,
                    size_of_image],
            outputs=[gen_video]
        )

    return sadtalker_interface

if __name__ == "__main__":
    demo = sadtalker_demo()
    demo.queue()
    demo.launch(share=True)
'''

with open('app.py', 'w') as f:
    f.write(app_content)

print("App.py перезаписан без TTS!")

/content/SadTalker
App.py перезаписан без TTS!


In [26]:
%cd /content/SadTalker

# Комментируем импорт TTS
!sed -i 's/from src.utils.text2speech import TTSTalker/# from src.utils.text2speech import TTSTalker/g' app.py

# Комментируем создание TTS объекта
!sed -i 's/self.tts_talker = TTSTalker()/# self.tts_talker = TTSTalker()/g' app.py

print("TTS отключен!")

/content/SadTalker
TTS отключен!


In [24]:
%cd /content/SadTalker

!wget -O checkpoints/epoch_20.pth https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/SadTalker_V0.0.2_256.safetensors
!wget -O checkpoints/facevid2vid_00189-model.pth.tar https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/facevid2vid_00189-model.pth.tar
!wget -O checkpoints/mapping_00109-model.pth.tar https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00109-model.pth.tar
!wget -O checkpoints/mapping_00229-model.pth.tar https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00229-model.pth.tar

print("Все модели скачаны!")

/content/SadTalker
--2025-11-11 17:29:00--  https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/SadTalker_V0.0.2_256.safetensors
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/569518584/93be550c-5100-467a-9ac3-994ddf04fb7e?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-11T18%3A15%3A33Z&rscd=attachment%3B+filename%3DSadTalker_V0.0.2_256.safetensors&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-11T17%3A14%3A59Z&ske=2025-11-11T18%3A15%3A33Z&sks=b&skv=2018-11-09&sig=D%2FHdz7aCcMls687Xq6tBxrc5ECMJi19jDa8w6JMlzH0%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2Mjg4NTc0MSwibmJmIjo

In [22]:
%cd /content/SadTalker

# Исправляем устаревший код
!sed -i 's/LandmarksType._2D/LandmarksType.TWO_D/g' src/face3d/extract_kp_videos.py
!sed -i 's/LandmarksType._3D/LandmarksType.THREE_D/g' src/face3d/extract_kp_videos.py

print("Код исправлен!")

/content/SadTalker
Код исправлен!


In [19]:
%cd /content/SadTalker
!ls examples/

/content/SadTalker
driven_audio  ref_video  source_image


In [9]:
%cd /content/SadTalker
!ls -la
!find . -name "*.py" | head -20

/content/SadTalker
total 92
drwxr-xr-x 9 root root  4096 Nov 11 16:30 .
drwxr-xr-x 1 root root  4096 Nov 11 16:26 ..
-rw-r--r-- 1 root root  6075 Nov 11 16:26 app.py
drwxr-xr-x 2 root root  4096 Nov 11 16:30 checkpoints
drwxr-xr-x 2 root root  4096 Nov 11 16:26 docs
drwxr-xr-x 5 root root  4096 Nov 11 16:26 examples
drwxr-xr-x 8 root root  4096 Nov 11 16:26 .git
-rw-r--r-- 1 root root  3165 Nov 11 16:26 .gitignore
-rw-r--r-- 1 root root  8622 Nov 11 16:26 inference.py
-rw-r--r-- 1 root root  1071 Nov 11 16:26 LICENSE
-rw-r--r-- 1 root root  6639 Nov 11 16:26 quick_demo.ipynb
-rw-r--r-- 1 root root 11860 Nov 11 16:26 README.md
-rw-r--r-- 1 root root   285 Nov 11 16:26 requirements3d.txt
-rw-r--r-- 1 root root   269 Nov 11 16:26 requirements.txt
drwxr-xr-x 7 root root  4096 Nov 11 16:29 SadTalker
drwxr-xr-x 2 root root  4096 Nov 11 16:26 scripts
drwxr-xr-x 8 root root  4096 Nov 11 16:26 src
./src/generate_batch.py
./src/test_audio2coeff.py
./src/audio2exp_models/networks.py
./src/audio2e